In [1]:
import pandas as pd 
import numpy as np 
import requests
import csv 
import re
from bs4 import BeautifulSoup

In [2]:
url="https://property.sulekha.com/apartments-flats-for-rent/pune#"
j=requests.get(url)
k=j.text
s=BeautifulSoup(k,'lxml')


In [3]:
print(s.prettify())

     <a href="https://www.sulekha.com/real-estate-developers/pune" title="Real Estate Developers in Pune">
            Real Estate Developers
           </a>
          </li>
          <li>
           <a href="https://www.sulekha.com/cottages/pune" title="Cottages in Pune">
            Cottages
           </a>
          </li>
          <li>
           <a href="https://www.sulekha.com/residential-packers-and-movers/pune" title="Residential Packers and Movers in Pune">
            Residential Packers and Movers
           </a>
          </li>
          <li>
           <a href="https://property.sulekha.com/pg-for-boys-ads/pune" title="Boys PG / Hostel in Pune">
            Boys PG / Hostel
           </a>
          </li>
          <li>
           <a href="https://property.sulekha.com/pg-for-girls-ads/pune" title="Girls PG / Hostel in Pune">
            Girls PG / Hostel
           </a>
          </li>
          <li>
           <a href="https://property.sulekha.com/male-roommates-ads/pune" 

In [4]:
def title_link(s):
    tit=[]
    link=[]
    sta=s.find_all('a',attrs={'class':"bizlinkurl"}) 
    for k in sta:
        tit.append(k.text)
        link.append(k['href'])
    d={
        'title_link':tit,
        'links':link
    }
    df=pd.DataFrame()
    return df

In [73]:
 address=[]
 k=[]
 sta=s.find_all('div',attrs={'class':"locality"}) 
 for a in sta:
     address.append(a.text)



 

In [76]:
len(location)

15

In [69]:

print(address)

['  Hadapsar, Pune    View Map ', '  EON IT PARK, Kharadi, Pune   Landmark: EON IT PARK  View Map ', '  Balewadi, Pune   Landmark: Near Jupiter Hospital  View Map ', '  Chinchwad Gaon, Pune   Landmark: talathi karyalya  View Map ', '  Shivthirth Nagar, Kothrud, Pune    View Map ', '  Magarpatta, Pune    View Map ', '  Hinjewadi Rajiv Gandhi Infotech Park, Hinjewadi, Pune   Landmark: Near Infosys Phase 2 Hinjewadi  View Map ', '  Kharadi, Pune   Landmark: Near eon it park  View Map ', '  Charholi Budruk, Pimpri Chinchwad   Landmark: sai mandir  View Map ', '  Moshi Pradhikaran, Pimpri Chinchwad   Landmark: Spineroad  View Map ', '  Wakad, Dange Chowk, Pimpri Chinchwad   Landmark: Wakad  View Map ', '  Near Ajmera school, Ajmera Complex, Pimpri Chinchwad    View Map ', '  Kharadi, Pune    View Map ', '  Kolte Patil Tuscan Estate., Kharadi, Pune   Landmark: Behind Radisson Hotel  View Map ', '  Matruchaya society, Kalyani Nagar, Pune   Landmark: Near gurunanak  View Map ']


In [42]:
location=[]
sta=s.find_all('a',attrs={'class':"view-on-map"})    
for a in sta:
    location.append(a['data-latlong'])

In [79]:
prices=[]
sta=s.find_all('span',attrs={'class':"deposit"})    
for i in sta:
    prices.append(i.text)
sta=s.find_all('span',attrs={'class':"Deposit"})    
for i in sta:
    prices.append(i.text)

In [63]:
len(prices[:])

15

In [64]:
prices[:]

[' Deposit:  30,000  ',
 ' Deposit:  75,000  ',
 ' Deposit:  60,000  ',
 ' Deposit:  25,000  ',
 ' Deposit:  40,000  ',
 ' Deposit:  80,000  ',
 ' Deposit:  60,000  ',
 ' Deposit:  50,000  ',
 ' Deposit:  40,000  ',
 ' Deposit:  25,000  ',
 ' Deposit:  50,000  ',
 ' Deposit:  20,000  ',
 ' Deposit:  12,000  ',
 ' Deposit:  105,000  ',
 ' Deposit:  16,000  ']

In [80]:
financial=[]
sta=s.find_all('span',attrs={'class':"price primary"})    
for i in sta:
    financial.append(i.text)

In [88]:
financial

['   13,960  / month   Deposit:  30,000   ',
 '   26,001  / month   Deposit:  75,000   ',
 '   22,000  / month   Deposit:  60,000   ',
 '   10,000  / month   Deposit:  25,000   ',
 '   14,000  / month   Deposit:  40,000   ',
 '   25,000  / month   Deposit:  80,000   ',
 '   21,000  / month   Deposit:  60,000   ',
 '   18,125  / month   Deposit:  50,000   ',
 '   14,000  / month   Deposit:  40,000   ',
 '   9,000  / month   Deposit:  25,000   ',
 '   18,000  / month   Deposit:  50,000   ',
 '   9,000  / month   Deposit:  20,000   ',
 '   6,500  / month   Deposit:  12,000   ',
 '   35,000  / month   Deposit:  105,000   ',
 '   11,000  / month   Deposit:  16,000   ']

In [90]:
ammenities=[]
sta=s.find_all('div',attrs={'class':"amenities flat"})   
for i in sta:
    ammenities.append(i.text)

In [91]:
spec_flat=[]
sta=s.find_all('div',attrs={'class':"spec flat"})   
for i in sta:
    spec_flat.append(i.text)

['No Car Parking 3rd (of 4 Floors) Semi Furnished Family Veg/Non Veg',
 '1 Car Parking 5th (of 10 Floors) Semi Furnished Family/Bachelor Veg/Non Veg',
 '1 Car Parking 1st (of 2 Floors) Semi Furnished Family/Bachelor',
 'No Car Parking 3rd (of 5 Floors) UnFurnished Family Veg/Non Veg',
 'No Car Parking 1st (of 3 Floors) UnFurnished Family/Bachelor',
 '1 Car Parking 5th (of 11 Floors) Fully Furnished Family/Bachelor',
 '1 Car Parking 23th (of 23 Floors) Semi Furnished Family/Bachelor Veg/Non Veg',
 '1 Car Parking 2nd (of 8 Floors) Semi Furnished Family/Bachelor Veg/Non Veg',
 '1 Car Parking 11th (of 12 Floors) UnFurnished Family Only Vegetarian',
 '1 Car Parking 2nd (of 4 Floors) Semi Furnished Family Veg/Non Veg',
 '1 Car Parking 8th (of 8 Floors) UnFurnished Family/Bachelor Veg/Non Veg',
 '1 Car Parking 1st (of 3 Floors) UnFurnished Family/Bachelor',
 'No Car Parking Ground Floor (of 4 Floors) Semi Furnished Family/Bachelor Veg/Non Veg',
 '1 Car Parking 5th (of 11 Floors) Fully Furnish

In [93]:
spec_flat_necce=[]
sta=s.find_all('div',attrs={'class':"spec icon primary"})   
for i in sta:
    spec_flat_necce.append(i.text)

In [ ]:
spec_flat_necce

In [96]:
dates_bro=[]
sta=s.find_all('div',attrs={'class':"posted"})   
for i in sta:
    dates_bro.append(i.text)


In [98]:
len(dates_bro)

15

In [101]:
the_text=[]
sta=s.find_all('span',attrs={'class':"listing-desc view-less"})   
for i in sta:
    the_text.append(i.text)

In [102]:
the_text

['East fag 1 BHK Flat for Rent Hadapsar, Pune . The Flat is a Semi Furnished property with Geysers, AC, Fans, Lights, Chimney, Modular Kitchen, Wardrobes. The builg has a total of 4 floor(s) and the Flat is on the 3rd Floor. The Flat is available for Rent at Rs.13960. Additional Information about the property: This Flat also has 1 balconies, Pooja Room, Study Room, Kids Room. The Flat offers you an unparalleled lig experience with amenities like Corporation water, Drage Facility, 24*7 Power Backup, Lifts, Gym, Security, ATM, Visitors Parg, Feng Shui / Vastu, Shopg Complex, Piped Gas, Inte, Tennis Court, Badton Court, Indoor Games, Play Ground, Yoga room, Medical Center, CCTV, Park, Community Hall, Service Lifts, Mtenance Staff, Jogg Track, R Water Harvesg, Sewage Treatment, Fire Safety, Laundromat, Cafe/Food Court, Day Care Center, Library, Basketball Court, Volleyball Court, Solar Power, Garbage Chute, Lobby Area, Plantation Avenue, Water treatment. With this property say goodbye to a